# Imports

In [15]:
import sys
 
# setting path
sys.path.append('..')

In [16]:
from dataset_builder import EEDatasetBuilder
import geemap
import ee

# Map

In [17]:
Map = geemap.Map(center=[-11.8673, -76.9544], zoom=2.5)
Map

Map(center=[-11.8673, -76.9544], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [18]:
# globe = ee.FeatureCollection('projects/ee-margauxmasson21-shapefiles/assets/world_rectangle')
# Map.addLayer(globe.geometry())

# Dataset builder

In [19]:
ee_dataset_builder = EEDatasetBuilder()

Make the biomass band with some filters

In [20]:
ee_dataset_builder.filtered_response_layer_from_raster(
    response_raster='Spawn_AGB_tCO2e',
    filter_dict={'forest_non_forest': {'date_range':['2010-01-01', '2010-12-31']},
                 'min_forest_age': {'age':40},
                # 'very_low_density_rural': {'year': 2010},
                # 'forest_loss': {'year': 2010, 'distance': 5000},
                # 'forest_gain' : {'distance': 5000},
                # 'roads': {'distance': 5000},
                # 'fire': {'year': 2010}
    }
)

Make covariates bands

In [21]:
ee_dataset_builder.spatial_covariates(covariates=['ecoregion', 'terrain'])

List band names of image

In [22]:
ee_dataset_builder.image.bandNames().getInfo()

['Spawn_AGB_tCO2e', 'BIOME_NUM', 'elevation', 'aspect', 'slope', 'hillshade']

In [23]:
ee_dataset_builder.image

So the image has the added bands; it basically seems to work.

Use the below code to make a plot and confirm results of mask match what is expected for both biomass and the features.

### Visualize in map

In [24]:
biomass_vis_param = { 
  'min': 0.0,
  'max': 600.0,
  'palette': ['d9f0a3', 'addd8e', '78c679', '41ab5d', '238443', '005a32']}

Map.addLayer(ee_dataset_builder.image.select('Spawn_AGB_tCO2e'), biomass_vis_param, 'Spawn Mature Forest Biomass', True)

In [25]:
biome_vis_param = { 
  'min': 0.0,
  'max': 50.0,
  'palette':  [
    "1c5510","659208","ae7d20","000065","bbcb35","009a18",
    "caffca","55eb49","65b2ff","0020ca","8ea228","ff9adf",
    "baff35","ffba9a","ffba35","f7ffca","e7e718","798649",
    "65ff9a","d29e96",
  ]};

Map.addLayer(ee_dataset_builder.image.select('BIOME_NUM'), biome_vis_param, 'Biome Num', True)

In [26]:
# asset_img = ee.Image('projects/ee-mmf-mature-forest-biomass/assets/mature_forest_2010_AGB_tco2_scale_100')
# Map.addLayer(asset_img, biomass_vis_param, 'Asset Mature Forest Biomass', True)

## Test other biomass rasters

In [27]:
ee_dataset_builder = EEDatasetBuilder()
ee_dataset_builder.filtered_response_layer_from_raster(
    response_raster='GEDI_Biomass_1km_tCO2',
    filter_dict={'forest_non_forest': {'date_range':['2010-01-01', '2010-12-31']},
                 'min_forest_age': {'age':40},
                # 'very_low_density_rural': {'year': 2010},
                # 'forest_loss': {'year': 2010, 'distance': 5000},
                # 'forest_gain' : {'distance': 5000},
                # 'roads': {'distance': 5000},
                # 'fire': {'year': 2010}
    }
)
ee_dataset_builder.spatial_covariates(covariates=['ecoregion', 'terrain'])
ee_dataset_builder.image.bandNames().getInfo()
Map.addLayer(ee_dataset_builder.image.select('GEDI_Biomass_1km_tCO2'), biomass_vis_param, 'GEDI Mature Forest Biomass', True)

In [28]:
ee_dataset_builder = EEDatasetBuilder()
ee_dataset_builder.filtered_response_layer_from_raster(
    response_raster='Walker_AGB_500m_tCO2',
    filter_dict={'forest_non_forest': {'date_range':['2010-01-01', '2010-12-31']},
                 'min_forest_age': {'age':40},
                # 'very_low_density_rural': {'year': 2010},
                # 'forest_loss': {'year': 2010, 'distance': 5000},
                # 'forest_gain' : {'distance': 5000},
                # 'roads': {'distance': 5000},
                # 'fire': {'year': 2010}
    }
)
ee_dataset_builder.spatial_covariates(covariates=['ecoregion', 'terrain'])
ee_dataset_builder.image.bandNames().getInfo()
Map.addLayer(ee_dataset_builder.image.select('Walker_AGB_500m_tCO2'), biomass_vis_param, 'Walker Mature Forest Biomass', True)

# Export samples CSV to GCP

Check progress in your GEE console: https://code.earthengine.google.com/tasks

In [13]:
# Gridded world shapefile asset in GEE
shp_asset_path = 'projects/ee-margauxmasson21-shapefiles/assets/latin_america_gridded_5000km2'

# This will take quite some time
gcp_bucket = 'eartshot-science-team'
gcp_folder_name = 'deforestation_risk'
scale = 30
numPixels = 2000
samples_folder_name = f'latin_america_gridded_5000km2_csv_scale{scale}_{numPixels}numPixels'

ee_dataset_builder.samples_csv_export(shp_asset_path,
                                      name_gcp_bucket=gcp_bucket,
                                      folder_in_gcp_bucket=gcp_folder_name + '/' + samples_folder_name,
                                      numPixels=numPixels, scale=scale)

# Export tiles to GCP

In [ ]:
################## Export inference tiles to GCP bucket ##################
shp_asset_path_tiles = 'projects/ee-margauxmasson21-shapefiles/assets/latin_america_gridded_10degrees'
tiles_folder_name = 'test_export_tiles_latinAm_gridded_10degrees'
maxPixels = 1e13

print(f'\nExport inference tiles using the shapefile: {shp_asset_path_tiles}...')
# This will take quite some time
ee_dataset_builder_inference.tiles_export(shp_asset_path_tiles,
                                    name_gcp_bucket=gcp_bucket,
                                    folder_in_gcp_bucket=gcp_folder_name + '/' + tiles_folder_name,
                                    maxPixels=maxPixels, scale=scale)
print('Inference tiles export done.\n')